In [1]:
# Pytorch
import torch, os, datetime, torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms 
from torchvision.utils import make_grid

#Plot
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns

import numpy as np
import random
import pandas as pd
import time
from sklearn.metrics import precision_score, recall_score, f1_score



Definindo uma seed padrão, para manter uma constância nos resultados, caso seja necessário rodar o projeto novamente

In [2]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Selecionando o dispositivo para rodar o projeto (Prefencialmente GPU)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# Carregando o DataSet


Iremos trabalhar com o valor de fechamento do BTC no dia

In [10]:
dataset = pd.read_csv('../data/Bitstamp_BTCUSD_d.csv')

num_linhas = int(len(dataset) * 0.2) # Pegando 20% do dataset

test = dataset.iloc[:num_linhas]
treino = dataset.iloc[num_linhas:]

In [13]:
print(f"Tamanho Dataset: {len(dataset)}\nTamanho conjunto de Treinamento: {len(treino)}\nTamanho Conjunto de Teste: {len(test)}")


Tamanho Dataset: 3477
Tamanho conjunto de Treinamento: 2782
Tamanho Conjunto de Teste: 695


In [15]:
test

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1717459200,2024-06-04 00:00:00,BTC/USD,68798.00,68868.00,68792.00,68857.00,0.628342,4.326574e+04
1,1717372800,2024-06-03 00:00:00,BTC/USD,67744.00,70268.00,67590.00,68801.00,2391.375924,1.645291e+08
2,1717286400,2024-06-02 00:00:00,BTC/USD,67722.00,68405.00,67281.00,67752.00,544.360624,3.688152e+07
3,1717200000,2024-06-01 00:00:00,BTC/USD,67513.00,67810.00,67387.00,67723.00,368.624037,2.496433e+07
4,1717113600,2024-05-31 00:00:00,BTC/USD,68330.00,69010.00,66624.00,67477.00,1880.059058,1.268607e+08
...,...,...,...,...,...,...,...,...,...
690,1657843200,2022-07-15 00:00:00,BTC/USD,20569.17,21186.35,20368.70,20835.39,1581.830129,3.295805e+07
691,1657756800,2022-07-14 00:00:00,BTC/USD,20225.24,20880.00,19608.47,20577.23,2574.758853,5.298141e+07
692,1657670400,2022-07-13 00:00:00,BTC/USD,19337.91,20367.88,18905.55,20232.35,3196.864138,6.468007e+07
693,1657584000,2022-07-12 00:00:00,BTC/USD,19948.55,20036.24,19237.42,19321.14,1738.030824,3.358074e+07


In [16]:
treino

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
695,1657411200,2022-07-10 00:00:00,BTC/USD,21581.98,21593.73,20629.99,20856.08,1.398918e+03,2.917595e+07
696,1657324800,2022-07-09 00:00:00,BTC/USD,21592.69,21955.18,21311.36,21588.35,8.653870e+02,1.868228e+07
697,1657238400,2022-07-08 00:00:00,BTC/USD,21620.85,22401.00,21164.07,21584.14,3.352948e+03,7.237049e+07
698,1657152000,2022-07-07 00:00:00,BTC/USD,20546.29,21847.25,20237.58,21643.56,3.422572e+03,7.407663e+07
699,1657065600,2022-07-06 00:00:00,BTC/USD,20164.65,20648.76,19750.16,20547.45,2.093382e+03,4.301366e+07
...,...,...,...,...,...,...,...,...,...
3472,1417478400,2014-12-02 00:00:00,BTC/USD,378.39,382.86,375.23,379.25,2.593576e+06,6.832530e+03
3473,1417392000,2014-12-01 00:00:00,BTC/USD,376.40,382.31,373.03,378.39,2.520662e+06,6.660560e+03
3474,1417305600,2014-11-30 00:00:00,BTC/USD,376.57,381.99,373.32,373.34,1.145567e+06,3.046330e+03
3475,1417219200,2014-11-29 00:00:00,BTC/USD,376.42,386.60,372.25,376.72,2.746157e+06,7.245190e+03


# Definindo a classe LSTM

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

# Definindo funções úteis

Função de Treinamento

In [ ]:
def train_model(model, criterion, optimizer, epochs):
    model.train().to(device)

    for epoch in range(epochs):
        running_loss = 0.0

        for inputs, labels in trainloader:
            (inputs, labels) = (inputs.to(device), labels.to(device))
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')
    
    print('Finished Training!')

Função de Teste do Modelo

In [ ]:
def test_model(model, criterion):
    model.eval().to(device)
    correct = 0
    total = 0
    test_loss = 0.0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = correct / total
    loss = test_loss / len(testloader)
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    return accuracy, loss, precision, recall, f1

Função de treinamento por Época

In [ ]:
def train_model_once(model, criterion, optimizer):
    model.train().to(device)

    running_loss = 0.0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
 
    return running_loss / len(trainloader)